# Message passing using Ray Queue

* [Message passing using Ray Queue](https://docs.ray.io/en/latest/ray-core/actors/actor-utils.html#message-passing-using-ray-queue)

> If you need to send data among many tasks or actors, you can use [ray.util.queue.Queue](https://docs.ray.io/en/latest/ray-core/api/doc/ray.util.queue.Queue.html).

* [Usage of ray.util.queue.Queue for data processing](https://github.com/ray-project/ray/issues/16007)

* [An Easy Way to Add Flow Control to Ray Applications](https://medium.com/ibm-data-ai/an-easy-way-to-add-flow-control-to-ray-applications-83c01c570565) (backpressure)

> Flow control is a computer networking term that basically means, “ensuring that the messages you send do not overwhelm the systems you are sending them through.” Applying flow control to your application’s interactions with a remote service means controlling things like the number of requests per second, the amount of data in flight at any time, or the amount of CPU load that you are putting on the remote service.
> 
> In this article, we’ll show you how to implement application-level flow control from inside a Ray application, using Ray’s own parallel processing APIs.

In [1]:
import ray
from ray.util.queue import Queue, Empty

In [8]:
ray.init()

2023-10-25 17:19:24,244	INFO worker.py:1553 -- Started a local Ray instance.


Python version:,3.9.13
Ray version:,2.3.0


In [9]:
# You can pass this object around to different tasks/actors
queue = Queue(maxsize=100)

In [10]:
@ray.remote
def consumer(id, queue):
    try:
        while True:
            next_item = queue.get(block=True, timeout=None)
            print(f"consumer {id} got work {next_item}")
    except Empty:
        print(f"{id} empty")
        pass

In [11]:
consumers = [
    consumer.remote(id, queue) 
    for id in range(2)
]

In [12]:
[queue.put(i) for i in range(10)]

[None, None, None, None, None, None, None, None, None, None]

(consumer pid=93649) consumer 1 got work 1
(consumer pid=93649) consumer 1 got work 3
(consumer pid=93649) consumer 1 got work 8
(consumer pid=93649) consumer 1 got work 9
(consumer pid=93650) consumer 0 got work 0
(consumer pid=93650) consumer 0 got work 2
(consumer pid=93650) consumer 0 got work 4
(consumer pid=93650) consumer 0 got work 5
(consumer pid=93650) consumer 0 got work 6
(consumer pid=93650) consumer 0 got work 7


In [13]:
ray.shutdown()